---
title: "Bike Map"
format: html
---



Bienvenue, voici la carte :


In [ ]:
import geopandas as gpd
import folium
import json
from shapely.geometry import Point
from IPython.display import IFrame

# Données
traffic_data_path = 'collecte/map/eco_comptage.json'
with open(traffic_data_path, 'r') as file:
    traffic_data = json.load(file)

# Coordonnées null enlevées
valid_traffic_data = [
    item for item in traffic_data 
    if 'coordinates' in item['location'] and item['location']['coordinates'][0] is not None and item['location']['coordinates'][1] is not None
]

# GeoDataFrame
traffic_gdf = gpd.GeoDataFrame(
    valid_traffic_data,
    geometry=[Point(data['location']['coordinates']) for data in valid_traffic_data],
    crs="EPSG:4326"
)

# Routes
routes_gdf = gpd.read_file('collecte/map/export.geojson')
routes_gdf = routes_gdf.to_crs(traffic_gdf.crs)

# Jointure
joined_gdf = gpd.sjoin_nearest(routes_gdf, traffic_gdf, how="inner", max_distance=100)

# Fonction pour choisir la couleur
def get_color(intensity):
    if intensity > 1000:
        return 'red'
    elif intensity > 500:
        return 'orange'
    else:
        return 'green'

# Création de la carte
map = folium.Map(location=[43.610769, 3.876716], zoom_start=13)

# Ajout des lignes sur la carte
for _, row in joined_gdf.iterrows():
    route_color = get_color(row['intensity'])
    folium.PolyLine(
        locations=[[p[1], p[0]] for p in list(row['geometry'].coords)],
        color=route_color,
        weight=5
    ).add_to(map)

# Sauvegarde temporaire et affichage dans Quarto
IFrame(src=map_path, width="100%", height="600")
